In [6]:
import pandas as pd
import numpy as np

from helpers import utils
from app.engine import routes

In [15]:
raw = (
    pd.read_csv(utils.get_processed_file('international-routes-pax.csv'))
    .assign(date = lambda x: pd.to_datetime(x.yearMonth, format='%Y%m', errors='coerce'))
    .assign(regionDest = lambda x: x.regionDest.str.replace('[^a-zA-Z]', ''))
)

In [16]:
def build_route_name(row):
    return '{} - {} ({})'.format(
        row.airportOrigin,
        row.airportDest,
        row.countryDest
    )
raw['route'] = raw.apply(build_route_name, axis=1)

In [24]:
raw.to_csv(utils.get_interim_file('international-routes-pax.csv'), index=False)

In [18]:
def agg(x):
    total = x.paxTotal.sum()
    scheduled = x.paxScheduled.sum()
    charter = x.paxCharter.sum()
    
    return pd.Series({
        'paxTotal': total,
        'paxScheduled': scheduled,
        'paxCharter': charter,
        'ratioScheduled': scheduled / total,
        'ratioCharter': charter / total,
        'numRoutes': x.route.nunique()
    })

df_region = (
    raw
    .groupby(['date', 'regionDest'])
    .apply(agg)
    .reset_index()
)
df_region.sample(5)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in long_scalars
  # This is added back by InteractiveShellApp.init_path()


,date,regionDest,paxTotal,paxScheduled,paxCharter,ratioScheduled,ratioCharter,numRoutes
2621,2011-07-01,OILRIGS,64933.0,0.0,64933.0,0.000000,1.000000,6.0
1081,2005-09-01,EASTAFRICA,68834.0,64542.0,4292.0,0.937647,0.062353,9.0
276,2002-07-01,CARIBBEANAREA,135718.0,111134.0,24584.0,0.818860,0.181140,31.0
3227,2013-11-01,CENTRALAMERICA,100002.0,38920.0,61082.0,0.389192,0.610808,18.0
879,2004-11-01,SOUTHERNAFRICA,162090.0,161814.0,276.0,0.998297,0.001703,6.0


In [27]:
raw.loc[lambda x: (x.regionDest == 'CANADA') & (x.airportDest == 'GOOSE BAY')]

,yearMonth,regionDest,countryDest,airportOrigin,airportDest,paxTotal,paxScheduled,paxCharter,paxTotalPrevMonth,paxScheduledPrevMonth,paxCharterPrevMonth,momTotalPctChange,date,route
69971,200512,CANADA,CANADA,GATWICK,GOOSE BAY,0,0,0,106,0,106,0.0,2005-12-01,GATWICK - GOOSE BAY (CANADA)
87314,200610,CANADA,CANADA,LUTON,GOOSE BAY,2,0,2,0,0,0,0.0,2006-10-01,LUTON - GOOSE BAY (CANADA)
93951,200702,CANADA,CANADA,LUTON,GOOSE BAY,45,0,45,0,0,0,0.0,2007-02-01,LUTON - GOOSE BAY (CANADA)
109162,200710,CANADA,CANADA,LUTON,GOOSE BAY,0,0,0,2,0,2,0.0,2007-10-01,LUTON - GOOSE BAY (CANADA)
116159,200802,CANADA,CANADA,LUTON,GOOSE BAY,0,0,0,45,0,45,0.0,2008-02-01,LUTON - GOOSE BAY (CANADA)
125465,200807,CANADA,CANADA,LUTON,GOOSE BAY,4,0,4,0,0,0,0.0,2008-07-01,LUTON - GOOSE BAY (CANADA)
127577,200808,CANADA,CANADA,STANSTED,GOOSE BAY,177,0,177,0,0,0,0.0,2008-08-01,STANSTED - GOOSE BAY (CANADA)
131722,200810,CANADA,CANADA,LUTON,GOOSE BAY,3,0,3,0,0,0,0.0,2008-10-01,LUTON - GOOSE BAY (CANADA)
147937,200907,CANADA,CANADA,LUTON,GOOSE BAY,0,0,0,4,0,4,0.0,2009-07-01,LUTON - GOOSE BAY (CANADA)
149956,200908,CANADA,CANADA,LUTON,GOOSE BAY,6,0,6,0,0,0,0.0,2009-08-01,LUTON - GOOSE BAY (CANADA)


In [23]:
(
    raw
    .groupby(['regionDest'])
    .agg({
        'date': [np.min, np.max]
    })
    .reset_index()
)

regionDest       date           
                                amin       amax
0    ATLANTICOCEANISLANDS 2001-06-01 2014-12-01
1             AUSTRALASIA 2001-06-01 2014-12-01
2                  CANADA 2001-06-01 2014-12-01
3           CARIBBEANAREA 2001-06-01 2014-12-01
4           CENTRALAFRICA 2001-06-01 2014-12-01
5          CENTRALAMERICA 2001-06-01 2014-12-01
6                  CYPRUS 2001-06-01 2004-12-01
7              EASTAFRICA 2001-06-01 2014-12-01
8           EASTERNEUROPE 2001-06-01 2004-12-01
9         EASTERNEUROPEEU 2005-01-01 2014-12-01
10     EASTERNEUROPEOTHER 2005-01-01 2014-12-01
11                FAREAST 2001-06-01 2014-12-01
12     INDIANOCEANISLANDS 2001-06-01 2014-12-01
13     INDIANSUBCONTINENT 2001-06-01 2014-12-01
14             MIDDLEEAST 2001-06-01 2014-12-01
15               NEAREAST 2001-06-01 2014-12-01
16            NORTHAFRICA 2001-06-01 2014-12-01
17                OILRIGS 2001-06-01 2014-12-01
18    PACIFICOCEANISLANDS 2005-03-01 2010-02-01
19           SOUTHAMERICA 2001-06-01 2014-12-01
20         SOUTHERNAFRICA 2001-06-01 2014-12-01
21  UNITEDSTATESOFAMERICA 2001-06-01 2014-12-01
22             WESTAFRICA 2001-06-01 2014-12-01
23       WESTERNEUROPEEEC 2001-06-01 2003-01-01
24        WESTERNEUROPEEU 2002-02-01 2014-12-01
25     WESTERNEUROPEOTHER 2001-06-01 2014-12-01

Looks like Eastern Europe splits into '-other' and '-eu' after 2004-12-01
Cyprus also ends 2004-12-01
- check if any notes on data changing, starting 2005-01-01

Pacific Ocean Islands runs only between 2005-03-01 and 2010-02-01

Western Europe EEC runs up to 2003-01-01
Western Europe EU runs from 2002-02-01
Western Europe Other is present for the entire period however